In [2]:
using POMDPs
using Distributions: Normal
using Random
import POMDPs: initialstate_distribution, actions, gen, discount, isterminal
Random.seed!(1);

In [51]:
struct human_goal_location
    x:: Int64
    y:: Int64
end

struct pedestrian_state
    x:: Int64
    y:: Int64
    goal:: human_goal_location
end

struct cart_state
    x:: Int64
    y:: Int64
    theta:: Int64
    v:: Int64
end    

struct observations
    observed_human_positions:: Array{human_goal_location}
end

struct SP_POMDP_state
    cart:: cart_state
    pedestrians:: Array{pedestrian_state}
    pedestrian_goals:: Array{human_goal_location}
    path_covered_index:: Int64
end

struct human_goal_probability
    distribution::Array{Float64}
end

function isgoalstate(s)
    cart_x = s.cart.x
    cart_y = s.cart.y
    cart_goal = m.cart_goal_position
    if(cart_goal.x == cart_x && cart_goal.y == cart_y)
        return true
    else
        return false
    end
end

ErrorException: invalid redefinition of constant SP_POMDP_state

In [10]:
g1 =  human_goal_location(4,5)
ps1 = pedestrian_state(1,1,g1)
ps2 = pedestrian_state(2,2,g1)
cs = cart_state(1,2,3,4)
pd = SP_POMDP_state(cs,[],[],1)
push!(pd.pedestrians,ps1)
push!(pd.pedestrians,ps2)
pd.pedestrians[2].goal

human_goal_location(4, 5)

In [52]:
mutable struct Speed_Planner_POMDP <: POMDPs.POMDP{SP_POMDP_state,Int,observations}
    discount_factor::Float64
    step_size::Int
    collision_threshold::Float64
    goal_reward::Int64
    max_cart_speed::Int64
    cart_goal_position::human_goal_location
    starting_cart_state::cart_state
    starting_human_states::Array{pedestrian_state}
    fixed_goal_locations::Array{human_goal_location}
    human_goals_prob_distribution::Array{human_goal_probability}
    astar_path::Array{Tuple{Int64,Int64},1}
    start_path_index::Int64
end
Speed_Planner_POMDP() = Speed_Planner_POMDP(0.9,1,2,100,5,)
discount(p::Speed_Planner_POMDP) = p.discount_factor
isterminal(::Speed_Planner_POMDP, s::SP_POMDP_state) = isgoalstate(s);

ErrorException: invalid redefinition of constant Speed_Planner_POMDP

In [ ]:
# mutable struct LightDark1D <: POMDPs.POMDP{Float64,Int,Int}
#     discount_factor::Float64
#     correct_r::Float64
#     incorrect_r::Float64
#     #step_size::Int
#     movement_cost::Float64
# end
# LightDark1D() = LightDark1D(0.9, 10, -10, 1, 0)
# discount(p::LightDark1D) = p.discount_factor
# isterminal(::LightDark1D, s::Float64) = isnan(s);

In [6]:
function POMDPs.gen(m::Speed_Planner_POMDP, s, a, rng)
    
    # transition model
    
    function calculate_theta(current_state, previous_state)
        theta = 0
        x_diff = current_state[1] - previous_state[1]
        y_diff = current_state[2] - previous_state[2]
        if x_diff != 0
            if x_diff < 0
                theta = 90
            else
                theta = 270
            end
        end
        if y_diff != 0
            if y_diff < 0
                theta = 0
            else
                theta = 180
            end
        end
        return theta
    end
    
    function update_human_state(human, human_goals, rng)
        goal = human.goal
        
        human_fixed_goals = copy(human_goals)
        deleteat!(human_fixed_goals, findall(x -> x==goal, human_fixed_goals)[1])
        
        rand_num = rand(rng)
        
        function move_human_towards_goal(human, goal)
            temp_human_x = human.x
            temp_human_y = human.y
            if temp_human_x < goal.x
                temp_human_x = temp_human_x + 1
            elseif temp_human_x > goal.x
                temp_human_x = temp_human_x - 1
            end

            if temp_human_y < goal.y
                temp_human_y = temp_human_y + 1
            elseif temp_human_y > goal.y
                temp_human_y = temp_human_y - 1
            end
            return pedestrian_state(temp_human_x, temp_human_y, goal)
        end
        
        if rand_num <= 0.7
            # move human towards goal
            new_human = move_human_towards_goal(human, goal)
        elseif rand_num > 0.7 && rand_num <= 0.8
            new_human = move_human_towards_goal(human, human_fixed_goals[1])
        elseif rand_num > 0.8 && rand_num <= 0.9
            new_human = move_human_towards_goal(human, human_fixed_goals[2])
        elseif rand_num > 0.9
            new_human = move_human_towards_goal(human, human_fixed_goals[3])
        end
        return new_human
    end

    new_pedestrians = []
    
    # action 0
    if a == 0
        # kart state +2 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v
        new_v = s.cart.v + a
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
    
    # action 1
    elseif a == 1
        # kart state +3 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v +1
        new_v = s.cart.v + a % 5
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
            
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    # action -1
    elseif a == -1
        # kart state +1 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v -1
        new_v = s.cart.v + a
        if new_v < 0
            new_v = 0
        end
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    end
    
    # update the state object
    sp = SP_POMDP_state(cart_new_state, new_pedestrians, s.pedestrian_goals, s.path, new_path_index)

    # observation model
    o = new_pedestrians
    #This is wrong^
    
    
    # reward model
    
    # collision reward
    function collision_reward(sp, coll_threshold)
        total_reward = 0
        cart_pose_x = sp.cart.x
        cart_pose_y = sp.cart.y
        for human in sp.pedestrians
            dist = ((human.x - cart_pose_x)^2 + (human.y - cart_pose_y)^2)^0.5
            if dist < coll_threshold
                total_reward = total_reward - 10
            end
        end
        return total_reward
    end
    
    # goal reward
    function goal_reward(sp, s, goal_state_reward)
        total_reward = -1
        cart_new_pose_x = sp.cart.x
        cart_new_pose_y = sp.cart.y
        
        cart_goal = sp.path[length(sp.path)]
        new_dist = ((cart_goal[1] - cart_new_pose_x)^2 + (cart_goal[1] - cart_new_pose_y)^2)^0.5
        
        cart_old_pose_x = sp.cart.x
        cart_old_pose_y = sp.cart.y
        old_dist = ((cart_goal[1] - cart_old_pose_x)^2 + (cart_goal[1] - cart_old_pose_y)^2)^0.5
        
        if new_dist < old_dist && new_dist != 0
            total_reward = goal_state_reward/new_dist
        elseif new_dist == 0
            total_reward = goal_state_reward
        end
        return total_reward
    end
    
    # speed reward
    function speed_reward(sp, max_speed)
        return (sp.cart.v - max_speeed)/max_speed
    end
    
    r = collision_reward(sp, m.collision_threshold) + goal_reward(sp, s, m.goal_reward) + speed_reward(sp, m.max_cart_speed)
    
    # create and return a NamedTuple
    return (sp=sp, o=o, r=r)
    
end

In [54]:
#Action Space for the POMDP
actions(::Speed_Planner_POMDP) = [-1, 0, 1] # Decelerate Maintain Accelerate

actions (generic function with 2 methods)

In [73]:
function initialstate_distribution(m::Speed_Planner_POMDP)
    initial_cart_state = m.starting_cart_state
    all_human_goal_locations = m.fixed_human_goal_locations
    initial_human_states = m.starting_human_states
    initial_path_start_index = m.start_path_index
    initial_human_goal_probability = m.human_goals_prob_distribution
    num_goals = length(all_human_goal_locations)
    
    all_256_possible_states = []
    all_256_probability_values = []
    
    for goal_human1_index in (1:num_goals)
        for goal_human2_index in (1:num_goals)
            for goal_human3_index in (1:num_goals)
                for goal_human4_index in (1:num_goals)
                    sampled_human1_sate = pedestrian_state(initial_human_states[1].x,initial_human_states[1].y,all_human_goal_locations[goal_human1_index])
                    sampled_human2_sate = pedestrian_state(initial_human_states[2].x,initial_human_states[2].y,all_human_goal_locations[goal_human2_index])
                    sampled_human3_sate = pedestrian_state(initial_human_states[3].x,initial_human_states[3].y,all_human_goal_locations[goal_human3_index])
                    sampled_human4_sate = pedestrian_state(initial_human_states[4].x,initial_human_states[4].y,all_human_goal_locations[goal_human4_index])
                    sampled_humans = [sampled_human1_state, sampled_human2_sate, sampled_human3_sate, sampled_human4_sate]                    
                    generated_state = SP_POMDP_state(initial_cart_state,sampled_humans,all_human_goal_locations,initial_path_start_index)
                    push!(all_256_possible_states,generated_state)
                    
                    human1_prob = initial_human_goal_probability[1].distribution[goal_human1_index]
                    human2_prob = initial_human_goal_probability[2].distribution[goal_human2_index]
                    human3_prob = initial_human_goal_probability[3].distribution[goal_human3_index]
                    human4_prob = initial_human_goal_probability[4].distribution[goal_human4_index]
                    probability_for_generated_state =  human1_prob*human2_prob*human3_prob*human4_prob
                    push!(all_256_probability_values,probability_for_generated_state)
                end
            end
        end
    end
                    
    return SparseCat(all_256_possible_states, all_256_probability_values)
end

initialstate_distribution (generic function with 1 method)

In [ ]:
# ?discount

In [ ]:
# @requirements_info GridWorld()

In [ ]:
# ?initialstate_distribution

In [ ]:
?isterminal

In [12]:

# initialstate_distribution(pomdp::LightDark1D) = Normal(2.0, 3.0);

In [45]:
d = Array{Tuple{Int64,Int64},1}

Array{Tuple{Int64,Int64},1}

In [46]:
d = [(1,2),(2,3),(3,4)]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 4)

In [47]:
typeof(c)

Array{Tuple{Int64,Int64},1}

In [49]:
d[2]

(2, 3)

In [58]:
actions(m::Speed_Planner_POMDP)

UndefVarError: UndefVarError: m not defined

In [64]:
x = 3
for i in (1:x)
    @show(i)
end

i = 1
i = 2
i = 3


In [65]:
length(d)

3

In [67]:
hg1 = human_goal_location(1,1)
p1 = pedestrian_state(1,1,hg1)
p2 = pedestrian_state(1,2,hg1)
lk = [p1,p2]

2-element Array{pedestrian_state,1}:
 pedestrian_state(1, 1, human_goal_location(1, 1))
 pedestrian_state(1, 2, human_goal_location(1, 1))

In [69]:
typeof(lk)

Array{pedestrian_state,1}

In [71]:
a= 1
b = 2
function lala(x,y)
    x = x+2
    @show(x)
    return x+y
end
@show(a)
@show(b)
c = lala(a,b)
@show(a)
@show(b)
@show(c)

a = 1
b = 2
x = 3
a = 1
b = 2
c = 5


5